# freeze 2d layer
* https://github.com/galidor/PyTorchPartialLayerFreezing/blob/main/partial_freezing.py

In [4]:
class Conv2d(nn.Module):
    def __init__(self):
        pass

In [17]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


import sys
sys.path.insert(0, "helper")
sys.path.insert(0, "/helper")
sys.path.insert(0, "./helper")
sys.path.insert(0, "../helper")
print(sys.path)

# own module
from visualisation.feature_map import *

import random


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.K = 100 
        self.L = 10 # last one
        self.num_of_bases = 1 # 3rd dim
        
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=0, dilation=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0, dilation=3)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=0, dilation=3)
        self.conv1x1 = nn.Conv2d(128, 10, kernel_size=1, stride=1, padding=0, dilation=1)
        
        self.fc3 = nn.Linear(10, 10)
        
        self.mish1 = torch.nn.Mish()
        self.mish2 = torch.nn.Mish()
        self.mish3 = torch.nn.Mish()
        self.mish1x1 = torch.nn.Mish()
        
        self.flat = nn.Flatten()
        
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        
        x = self.conv1(x)
        x = self.mish1(x)
        
        x = self.conv2(x)
        x = self.mish2(x)
        
        x = self.conv3(x)
        x = self.mish3(x)
        
        x = self.conv1x1(x)
        x = self.mish1x1(x)
        
        x = F.avg_pool2d(x, kernel_size=x.size()[2:])
        
        x = x.reshape(x.size(0), -1)
        x = self.fc3(x)
        
        x = self.sigmoid(x)
        
        return x





['../helper', './helper', '/helper', 'helper', '../helper', './helper', '/helper', 'helper', '../helper', './helper', '/helper', 'helper', 'C:\\Users\\Christina\\Documents\\datasceyence\\examples', 'C:\\Users\\Christina\\anaconda3\\envs\\chrisy\\python39.zip', 'C:\\Users\\Christina\\anaconda3\\envs\\chrisy\\DLLs', 'C:\\Users\\Christina\\anaconda3\\envs\\chrisy\\lib', 'C:\\Users\\Christina\\anaconda3\\envs\\chrisy', '', 'C:\\Users\\Christina\\anaconda3\\envs\\chrisy\\lib\\site-packages', 'C:\\Users\\Christina\\anaconda3\\envs\\chrisy\\lib\\site-packages\\pixelssl-0.1.4-py3.9.egg', 'C:\\Users\\Christina\\anaconda3\\envs\\chrisy\\lib\\site-packages\\win32', 'C:\\Users\\Christina\\anaconda3\\envs\\chrisy\\lib\\site-packages\\win32\\lib', 'C:\\Users\\Christina\\anaconda3\\envs\\chrisy\\lib\\site-packages\\Pythonwin']


In [18]:
def freeze_conv2d_params(layer, weight_indices, bias_indices=None, weight_hook_handle=None, bias_hook_handle=None):
    if weight_hook_handle is not None:
        weight_hook_handle.remove()
    if bias_hook_handle is not None:
        bias_hook_handle.remove()

    if (weight_indices == [] or weight_indices is None) and (bias_indices == [] or bias_indices is None):
        return

    if bias_indices is None:
        bias_indices = weight_indices

    if not (isinstance(layer, Conv2d) or isinstance(layer, nn.Conv2d)):
        raise ValueError("layer must be a valid Conv2d layer")

    if max(weight_indices) >= layer.weight.shape[0]:
        raise IndexError("weight_indices must be less than the number output channels")

    if layer.bias is not None:
        if max(bias_indices) >= layer.bias.shape[0]:
            raise IndexError("bias_indices must be less than the number output channels")

    def freezing_hook_weight_full(grad, weight_multiplier):
        return grad * weight_multiplier

    def freezing_hook_bias_full(grad, bias_multiplier):
        return grad * bias_multiplier

    weight_multiplier = torch.ones(layer.weight.shape[0]).to(layer.weight.device)
    weight_multiplier[weight_indices] = 0
    weight_multiplier = weight_multiplier.view(-1, 1, 1, 1)
    freezing_hook_weight = lambda grad: freezing_hook_weight_full(grad, weight_multiplier)
    weight_hook_handle = layer.weight.register_hook(freezing_hook_weight)

    if layer.bias is not None:
        bias_multiplier = torch.ones(layer.weight.shape[0]).to(layer.bias.device)
        bias_multiplier[bias_indices] = 0
        freezing_hook_bias = lambda grad: freezing_hook_bias_full(grad, bias_multiplier)
        bias_hook_handle = layer.bias.register_hook(freezing_hook_bias)
    else:
        bias_hook_handle = None

    return weight_hook_handle, bias_hook_handle

In [19]:
net = Net()
    
device = 'cuda:0' if next(net.parameters()).is_cuda else 'cpu'

indices = [2, 3, 4, 5]
for module in net.modules():
    if isinstance(module, Conv2d) or isinstance(module, nn.Conv2d):
        print("is instance", module)
        freeze_conv2d_params(module, indices)

init_weights = net.conv1x1.weight.clone()
init_biases = net.conv1x1.bias.clone()

optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
random_input2d = torch.randn((1, 1, 32, 32)).to(device)

net.train()
optimizer.zero_grad()
output = net(random_input2d)
loss = F.cross_entropy(output, torch.tensor([5]).to(device))
loss.backward(create_graph=True)
if (net.conv1x1.bias.grad[indices] == torch.zeros_like(net.conv1x1.bias.grad[indices])).all() and \
   (net.conv1x1.weight.grad[indices, :, :, :] == torch.zeros_like(net.conv1x1.weight.grad[indices, :, :, :])).all():
    print("Conv2d grads passed{:}".format(' on CUDA' if device == 'cuda:0' else ''))
else:
    print("Conv2d grads failed{:}".format(' on CUDA' if device == 'cuda:0' else ''))
optimizer.step()

if (net.conv1x1.bias == init_biases)[indices].all() and (net.conv1x1.weight == init_weights)[indices].all():
    print("Conv2d frozen weights and biases passed{:}".format(' on CUDA' if device == 'cuda:0' else ''))
else:
    print("Conv2d frozen weights and biases failed{:}".format(' on CUDA' if device == 'cuda:0' else ''))


is instance Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), dilation=(3, 3))
is instance Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), dilation=(3, 3))
is instance Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), dilation=(3, 3))
is instance Conv2d(128, 10, kernel_size=(1, 1), stride=(1, 1))
Conv2d grads passed
Conv2d frozen weights and biases passed


In [20]:
net.conv1x1.weight

Parameter containing:
tensor([[[[ 0.0627]],

         [[-0.0126]],

         [[-0.0252]],

         ...,

         [[ 0.0054]],

         [[ 0.0637]],

         [[-0.0825]]],


        [[[ 0.0109]],

         [[ 0.0529]],

         [[-0.0795]],

         ...,

         [[ 0.0591]],

         [[-0.0128]],

         [[ 0.0613]]],


        [[[ 0.0810]],

         [[-0.0403]],

         [[ 0.0841]],

         ...,

         [[ 0.0565]],

         [[-0.0606]],

         [[ 0.0688]]],


        ...,


        [[[-0.0762]],

         [[-0.0401]],

         [[-0.0374]],

         ...,

         [[ 0.0009]],

         [[ 0.0382]],

         [[ 0.0635]]],


        [[[ 0.0813]],

         [[-0.0008]],

         [[-0.0214]],

         ...,

         [[ 0.0729]],

         [[ 0.0286]],

         [[ 0.0454]]],


        [[[-0.0055]],

         [[ 0.0606]],

         [[-0.0571]],

         ...,

         [[-0.0520]],

         [[ 0.0095]],

         [[ 0.0293]]]], requires_grad=True)

In [21]:
for i in net.modules():
    print(i.__class__.__name__)

Net
Conv2d
Conv2d
Conv2d
Conv2d
Linear
Mish
Mish
Mish
Mish
Flatten
Sigmoid


In [16]:
for name, module in net.named_modules():
    # print(module)
    if isinstance(module, Conv2d) or isinstance(module, nn.Conv2d):
        print(name, module.weight.grad)


conv1 tensor([[[[ 9.7991e-06,  2.7823e-05,  6.0301e-05],
          [-9.7006e-05, -9.8383e-05,  1.1868e-04],
          [-1.6725e-04, -5.5329e-05,  4.5236e-05]]],


        [[[-2.2441e-04, -6.1382e-05,  2.1444e-04],
          [ 6.3723e-06,  8.5212e-05,  2.5682e-04],
          [ 1.3520e-04, -1.4926e-04,  4.1073e-06]]],


        [[[-0.0000e+00, -0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00, -0.0000e+00],
          [ 0.0000e+00,  0.0000e+00, -0.0000e+00]]],


        [[[ 0.0000e+00,  0.0000e+00, -0.0000e+00],
          [ 0.0000e+00, -0.0000e+00, -0.0000e+00],
          [ 0.0000e+00, -0.0000e+00, -0.0000e+00]]],


        [[[-0.0000e+00, -0.0000e+00, -0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-0.0000e+00, -0.0000e+00,  0.0000e+00]]],


        [[[-0.0000e+00, -0.0000e+00,  0.0000e+00],
          [-0.0000e+00, -0.0000e+00, -0.0000e+00],
          [-0.0000e+00, -0.0000e+00,  0.0000e+00]]],


        [[[-1.3346e-04, -9.0026e-05, -9.3098e-05],
 